In [1]:
%%capture
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import numpy as np
import scipy as sp

In [2]:
with open('./data/aan_full.json') as f:
    full_set = json.load(f)

In [3]:
corpus = [ref['title'] + ' ' + ref['abstract'] for ref in full_set]
vectorizer = TfidfVectorizer(stop_words='english').fit(corpus)
print(len(vectorizer.get_feature_names()))

82950


In [4]:
for ref in tqdm(full_set):
    document = [ref['title'] + ' ' + ref['abstract']]
    ref['tfidf_vector'] = vectorizer.transform(document)

In [5]:
full_set_dict = dict([(paper['id'], paper) for paper in full_set])

In [6]:
paper_tfidf_vectors = [paper['tfidf_vector'] for paper in full_set]
paper_ids = [paper['id'] for paper in full_set]

paper_tfidf_array = sp.sparse.vstack(paper_tfidf_vectors)

# aan_test_single.json

In [7]:
with open('./data/aan_test_single.json') as f:
    test_set = json.load(f)

In [8]:
input_tfidf_vectors = [full_set_dict[paper['id']]['tfidf_vector'] for paper in test_set]
input_ids = [paper['id'] for paper in test_set]

input_tfidf_array = sp.sparse.vstack(input_tfidf_vectors)

score_array = cosine_similarity(input_tfidf_array, paper_tfidf_array)

In [9]:
results = []

for input_paper_id, scores in tqdm(zip(input_ids, score_array.tolist())):
    result = {}
    result['input'] = [input_paper_id]
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] != input_paper_id]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [10]:
with open('./results/single_base_tfidfcosine.json', 'w') as f:
    json.dump(results, f)

# aan_test_triplet.json

In [9]:
with open('./data/aan_test_triplet.json') as f:
    test_set = json.load(f)

In [12]:
input_documents = []
for input_papers in test_set:
    input_document = ''
    
    for input_paper in input_papers:
        input_document += input_paper['title'] + ' ' + input_paper['abstract']
    
    input_documents.append(input_document)

input_tfidf_array = vectorizer.transform(input_documents)

score_array = cosine_similarity(input_tfidf_array, paper_tfidf_array)

In [13]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [14]:
with open('./results/triplet_base_tfidfcosine_concat.json', 'w') as f:
    json.dump(results, f)

In [10]:
partial_score_arrays = []
for i in tqdm(range(3)):
    partial_input_ids = [input_papers[i]['id'] for input_papers in test_set]
    input_tfidf_vectors = [full_set_dict[paper_id]['tfidf_vector'] for paper_id in partial_input_ids]
    
    input_tfidf_array = sp.sparse.vstack(input_tfidf_vectors)

    partial_score_arrays.append(cosine_similarity(input_tfidf_array, paper_tfidf_array))

score_array = sum(partial_score_arrays)

In [17]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [18]:
with open('./results/triplet_base_tfidfcosine_sum.json', 'w') as f:
    json.dump(results, f)

# aan_test_tripletfromref.json

In [7]:
with open('./data/aan_test_tripletfromref.json') as f:
    test_set = json.load(f)

In [8]:
input_documents = []
for input_papers in test_set:
    input_document = ''
    
    for input_paper in input_papers:
        input_document += input_paper['title'] + ' ' + input_paper['abstract']
    
    input_documents.append(input_document)

input_tfidf_array = vectorizer.transform(input_documents)

score_array = cosine_similarity(input_tfidf_array, paper_tfidf_array)

In [9]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [10]:
with open('./results/tripletfromref_base_tfidfcosine_concat.json', 'w') as f:
    json.dump(results, f)

In [21]:
partial_score_arrays = []
for i in tqdm(range(3)):
    partial_input_ids = [input_papers[i]['id'] for input_papers in test_set]
    input_tfidf_vectors = [full_set_dict[paper_id]['tfidf_vector'] for paper_id in partial_input_ids]
    
    input_tfidf_array = sp.sparse.vstack(input_tfidf_vectors)

    partial_score_arrays.append(cosine_similarity(input_tfidf_array, paper_tfidf_array))

score_array = sum(partial_score_arrays)

In [22]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [23]:
with open('./results/tripletfromref_base_tfidfcosine_sum.json', 'w') as f:
    json.dump(results, f)